In [1]:
skip_download = True  # If True, skip the download step
skip_transform = True  # If True, skip the transform step
skip_tracing = True  # If True, skip the tracing step

In [2]:
import os

if not skip_tracing:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_PROJECT"] = "Tx Generator"

In [3]:
from case import get_retriever

retriever = get_retriever()

In [4]:
retriever.invoke("Stake ETH with Lido and deposit to Eigenpie")

[Document(metadata={'case_id': 'eigenpie_steth', 'seq_num': 55, 'source': 'eigenpie/index.ts', 'steps': "[{'description': 'Stake ETH to Lido'}, {'description': 'Approve stETH to Eigenpie'}, {'description': 'Stake stETH to Eigenpie'}]", 'total_steps': 3}, page_content='A batch case for staking ETH to Lido and then restaking stETH to Eigenpie.'),
 Document(metadata={'case_id': 'stake_eth', 'seq_num': 95, 'source': 'stake-eth/stake.ts', 'steps': "[{'description': 'Stake into Lido'}, {'description': 'Stake into RocketPool'}, {'description': 'Swap ETH for staked token using 1inch'}, {'description': 'Stake into StakeWise'}, {'description': 'Stake into Stader'}, {'description': 'Stake into EtherFi'}, {'description': 'Stake into Renzo'}, {'description': 'Stake into Frax Finance'}, {'description': 'Stake into Swell'}, {'description': 'Stake into Redacted Finance'}, {'description': 'Stake into another protocol'}, {'description': 'Stake into another protocol'}, {'description': 'Stake into another

In [5]:
retriever.invoke("Swap 10 USDC to USDT")

[Document(metadata={'case_id': 'etf_ethereum_top10', 'seq_num': 57, 'source': 'etf-ethereum-top10/index.ts', 'steps': "[{'description': 'Approve USDT for 1inch.'}, {'description': 'Swap USDT to Top1 token.'}, {'description': 'Swap USDT to Top2 token.'}, {'description': 'Swap USDT to Top3 token.'}, {'description': 'Swap USDT to Top4 token.'}, {'description': 'Swap USDT to Top5 token.'}, {'description': 'Swap USDT to Top6 token.'}, {'description': 'Swap USDT to Top7 token.'}, {'description': 'Swap USDT to Top8 token.'}, {'description': 'Swap USDT to Top9 token.'}, {'description': 'Swap USDT to Top10 token.'}]", 'total_steps': 11}, page_content='Swap USDT to the top 10 tokens on Ethereum using 1inch.'),
 Document(metadata={'case_id': 'etf_base_meme_top10', 'seq_num': 56, 'source': 'etf-basememe-top10/index.ts', 'steps': "[{'description': 'Approve USDC for 1inch.'}, {'description': 'Swap USDC to Top1 token.'}, {'description': 'Swap USDC to Top2 token.'}, {'description': 'Swap USDC to Top3 

In [6]:
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from utils.model_selector import get_chat_model
from models.case import BatchCase


system_prompt = """
You are an expert case generator specializing in EVM blockchain transactions. 
Your role is to transform the user's intent into a set of actionable steps.

Guidelines:
------------
1. Ensure the output is clear, concise, and logically ordered.
2. If the user's intent is unclear, refine it as needed to generate a coherent set of steps.
3. Do not include any steps that require user input or feedback (e.g., asking the user to review received assets).
4. Only generate steps directly relevant to EVM blockchain transactions.
5. Reference the provided context when necessary.

Context:
------------
{context}
------------"""

user_intent = "{description}"

prompt = ChatPromptTemplate.from_messages([system_prompt, user_intent])

retriever = get_retriever()

model = get_chat_model().model


def format_docs(docs):
    return "\n\n".join(str(doc.metadata) for doc in docs)


# Chain
rag_chain = (
    {"context": retriever | format_docs, "description": RunnablePassthrough()}
    | prompt
    | model.with_structured_output(BatchCase)
)

In [7]:
question = "Stake 0.03 ETH with Lido and deposit to Eigenpie"
result = rag_chain.invoke(question)
print(result.json())

{"id": "stake_eth_lido_eigenpie", "description": "Stake 0.03 ETH with Lido and deposit to Eigenpie", "total_steps": 3, "steps": [{"description": "Stake 0.03 ETH to Lido."}, {"description": "Approve stETH to Eigenpie."}, {"description": "Stake stETH to Eigenpie."}]}


In [8]:
question = "Stake 2 ETH with Lido and deposit to Eigenpie"
result = rag_chain.invoke(question)
print(result.json())

{"id": "stake_2_eth_lido_eigenpie", "description": "Stake 2 ETH with Lido and deposit to Eigenpie", "total_steps": 3, "steps": [{"description": "Stake 2 ETH to Lido."}, {"description": "Approve stETH to Eigenpie."}, {"description": "Stake stETH to Eigenpie."}]}


In [9]:
question = "Swap 100 USDT for USDC on Uniswap"
result = rag_chain.invoke(question)
print(result.json())

{"id": "swap_usdt_to_usdc", "description": "Swap 100 USDT for USDC on Uniswap", "total_steps": 3, "steps": [{"description": "Approve 100 USDT for Uniswap."}, {"description": "Swap 100 USDT for USDC on Uniswap."}, {"description": "Confirm the transaction on the blockchain."}]}


In [10]:
question = "Swap 0.001 ETH for USDC on Uniswap"
result = rag_chain.invoke(question)
print(result.json())

{"id": "swap_eth_for_usdc_uniswap", "description": "Swap ETH for USDC on Uniswap", "total_steps": 1, "steps": [{"description": "Swap 0.001 ETH for USDC on Uniswap"}]}


In [11]:
question = """Yield Farming with Multiple Protocols
	Step 1: Deposit ETH into a AAVE to earn interest.
	Step 2: Use aToken (aETH) as collateral to borrow DAI.
	Step 3: Deposit the borrowed DAI into Yearn Finance.
	Step 4: Use the Yearn vault tokens in a liquidity pool on Uniswap to earn trading fees."""
result = rag_chain.invoke(question)
print(result.json())

{"id": "yield_farming_multiple_protocols", "description": "Yield Farming with Multiple Protocols", "total_steps": 4, "steps": [{"description": "Deposit ETH into AAVE to earn interest."}, {"description": "Use aToken (aETH) as collateral to borrow DAI."}, {"description": "Deposit the borrowed DAI into Yearn Finance."}, {"description": "Use the Yearn vault tokens in a liquidity pool on Uniswap to earn trading fees."}]}
